# Make Vt Datasets
Vt = F(W,L,Vgs,Vds,T,Ids) (Ids is optional)

In saturation point, Vt = Vgs-Vds.\
Let's assume we find saturation point by finding elbow point.\
For each (W,L,T,Vgs) dataset, we can find the elbow point from (Vds, Ids) curve.

## Briefly look data structure

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [12]:
sample_data = pd.read_csv('./data/W10L5T25.csv')
sample_data.head()

,Vds,Vgs,Ids,W,L,T
0,0.1,-20.0,7.510100e-14,10,5,25
1,0.1,-19.8,7.510100e-14,10,5,25
2,0.1,-19.6,7.510100e-14,10,5,25
3,0.1,-19.4,7.510100e-14,10,5,25
4,0.1,-19.2,7.510100e-14,10,5,25


In [13]:
sample_data_for_vds_ids = sample_data[['Vgs', 'Vds', 'Ids']]
sample_data_for_vds_ids.head()

,Vgs,Vds,Ids
0,-20.0,0.1,7.510100e-14
1,-19.8,0.1,7.510100e-14
2,-19.6,0.1,7.510100e-14
3,-19.4,0.1,7.510100e-14
4,-19.2,0.1,7.510100e-14


In [14]:
len(sample_data_for_vds_ids['Vgs'].unique())

201

## Find saturation point

Reference: https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points

In [15]:
saturation_point_dict = {}
saturation_point_list = []
vgs_index = sample_data_for_vds_ids['Vgs'].unique()
for vgs in vgs_index:
    vds_ids_part = sample_data_for_vds_ids.query('Vgs=='+str(vgs))
    p1 = vds_ids_part.query('Vds==0.1')[['Ids', 'Vds']].values
    p2 = vds_ids_part.query('Vds==30')[['Ids', 'Vds']].values

    max_dist = -99
    for vds in vds_ids_part['Vds'].unique():
        p3 = vds_ids_part.query('Vds=='+str(vds))[['Ids', 'Vds']].values
        current_dist = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
        if current_dist > max_dist:
            max_dist = current_dist
            elbow_point = p3
    saturation_point_dict[vgs] = elbow_point[0][1]
    # Vt = F(W,L,T,Vgs,Vds,Ids)
    saturation_point_list.append(((vgs-elbow_point[0][1]), '10', '5', '25', vgs, elbow_point[0][1], elbow_point[0][0]))

In [16]:
len(saturation_point_list)

201

In [17]:
saturation_point_list[0:5]

[(-21.0, '10', '5', '25', -20.0, 1.0, 7.6666e-14),
 (-20.8, '10', '5', '25', -19.8, 1.0, 7.6666e-14),
 (-20.6, '10', '5', '25', -19.6, 1.0, 7.6666e-14),
 (-20.4, '10', '5', '25', -19.4, 1.0, 7.6666e-14),
 (-20.2, '10', '5', '25', -19.2, 1.0, 7.6666e-14)]

In [18]:
# Make above function as method
def get_saturation_point_list(width, length, temperature):
    saturation_point_list = []
    data = pd.read_csv('./data/'+ 'W' + str(width) + 'L' + str(length) + 'T' + str(temperature) + '.csv')
    data_for_vds_ids = data[['Vgs', 'Vds', 'Ids']]
    vgs_index = data_for_vds_ids['Vgs'].unique()
    for vgs in vgs_index:
        vds_ids_part = data_for_vds_ids.query('Vgs=='+str(vgs))
        p1 = vds_ids_part.query('Vds==0.1')[['Ids', 'Vds']].values
        p2 = vds_ids_part.query('Vds==30')[['Ids', 'Vds']].values

        max_dist = -99
        for vds in vds_ids_part['Vds'].unique():
            p3 = vds_ids_part.query('Vds=='+str(vds))[['Ids', 'Vds']].values
            current_dist = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
            if current_dist > max_dist:
                max_dist = current_dist
                elbow_point = p3
        # Vt = F(W,L,T,Vgs,Vds,Ids)
        saturation_point_list.append([(vgs-elbow_point[0][1]), width, length, temperature, vgs, elbow_point[0][1], elbow_point[0][0]])
    return saturation_point_list

## Get Vt dataset
At saturation point,\
Save Vt(Vgs-Vds) = F(W,L,T,Vgs,Vds,Ids) as appropriate format.

### 1) List all cases of W,L,T

In [19]:
WIDTHS = [10, 20, 35, 50, 100, 1000]
LENGTHS = [5, 8, 10, 12]
TEMPS = [0, 25, 80]

## 2) Call method and get all saturation point list from whole datasets

In [20]:
total_saturation_point_list = []
for width in WIDTHS:
    for length in LENGTHS:
        for temperature in TEMPS:
            total_saturation_point_list += get_saturation_point_list(width, length, temperature)

len(total_saturation_point_list)

14472

In [22]:
vt_dataframe = pd.DataFrame(total_saturation_point_list, columns=['Vt', 'W', 'L', 'T', 'Vgs', 'Vds', 'Ids'])
vt_dataframe.head()

,Vt,W,L,T,Vgs,Vds,Ids
0,-21.0,10,5,0,-20.0,1.0,9.562200e-15
1,-20.8,10,5,0,-19.8,1.0,9.562200e-15
2,-20.6,10,5,0,-19.6,1.0,9.562200e-15
3,-20.4,10,5,0,-19.4,1.0,9.562200e-15
4,-20.2,10,5,0,-19.2,1.0,9.562200e-15


## 3) Save Vt dataset dataframe into CSV

In [25]:
vt_dataframe.to_csv('./data_vt/vt_dataset.csv', index=False)